## Imports

In [1]:
from cp import *

%load_ext autoreload
%autoreload 2


## CUDA

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
!which nvcc

/home/michael.me/miniconda3/envs/py37/bin/nvcc


In [4]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [5]:
# import os
# os.chdir('drive/MyDrive/Master/Reliability in ML/dynens')

In [6]:
!ls

cp.py		   images	   output	related		  venv
data		   jupyter-lab.sh  __pycache__	requirements.txt
experiments.ipynb  model	   README.md	script


In [7]:
# !pip install -r requirements.txt

In [8]:
# !sudo apt-get --purge remove cuda nvidia* libnvidia-*
# !sudo dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !sudo apt-get remove cuda-*
# !sudo apt autoremove
# !sudo apt-get update

In [9]:
# !sudo dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb

In [10]:
# !ls /var

In [11]:
# !sudo apt-key add /var/cuda-repo-10-0-local-10.0.130-410.48/7fa2af80.pub
# !sudo apt-get update

In [12]:
# !sudo apt-get -y install cuda-10-0

In [13]:
!nvidia-smi

Thu Jul  1 16:19:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 33%   34C    P0    28W / 250W |      0MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Experiments

### Data Reader

In [14]:
# %tensorflow_version 2.x  
import tensorflow as tf  
print(tf.__version__)
tf.test.gpu_device_name()  

2021-07-01 16:19:24.646797: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


2.5.0


2021-07-01 16:19:26.687480: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'/device:GPU:0'

2021-07-01 16:19:26.689100: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-01 16:19:28.130573: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:1e:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-07-01 16:19:28.130632: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-01 16:19:28.134401: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-01 16:19:28.134501: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-07-01 16:19:28.136334: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [15]:
"""
methods to read polarity data 
and make it avaiable for fasttext classification
Done by Tere
Nov 7, 2019
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow.keras.preprocessing.sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# from tensorflow.contrib import learn



"""
text processing
and split text/train
"""
def prepare_text(x,num_words,max_len):
    
    print ("to tokenize")
    #split test and training
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(x)
    
    print ("unique tokens:",len(tokenizer.word_index))
    x = tokenizer.texts_to_sequences(x)
    x = pad_sequences(x, maxlen=max_len,padding='post', truncating='pre')
    
    return x
    
def read_yahoo_files(file_train, file_test,file_val):
    
    names  = ["class", "questionlabel","questionContent","answer"]
    
    df_train   = pd.read_csv(file_train,names=names)
    df_test    = pd.read_csv(file_test,names=names)
    df_val     = pd.read_csv(file_val,names=names)
    
    train_len = len(df_train)
    test_len  = len(df_test)
    val_len   = len(df_val)
    
    print ("train len =",train_len)
    print ("test len  =",test_len)
    print ("val len   =",val_len)
    
    df = pd.concat([df_train,df_test,df_val])
    
    x_text = df["questionlabel"].astype(str) +" "+df["questionContent"].astype(str) +" "+df["answer"].astype(str)
    #x_text = df["answer"].astype(str)
    x = x_text.tolist() #np.array(x_text)#x_text.tolist(
    df_y = df["class"]
    y = pd.get_dummies(df_y,columns=['class']).values 
    return x,y, train_len,test_len, val_len
                     
def load_data_yahoo_ans(src_path, max_words=20000,max_len=1000):
    fileTrain = src_path + "train.csv"
    fileTest  = src_path + "test.csv"
    fileVal  = src_path + "val.csv"

    print ("to read")
    x,y,train_len,test_len,val_len = read_yahoo_files(fileTrain,fileTest,fileVal)
    
    print ("to process")
    x = prepare_text(x,max_words,max_len)
    
    print ("to split") 
    x_train = x[0:train_len,:]
    y_train = y[0:train_len,:]
    
    x_test  = x[train_len:(train_len+test_len),:]
    y_test  = y[train_len:(train_len+test_len),:]

    x_val  = x[(train_len+test_len):(train_len+test_len+val_len),:]
    y_val  = y[(train_len+test_len):(train_len+test_len+val_len),:]

    print ("New train SHAPE")
    print (x_train.shape)
    print (y_train.shape)
    
    print ("New test SHAPE")
    print (x_test.shape)
    print (y_test.shape)
    
    print ("New train SHAPE")
    print (x_test.shape)
    print (y_test.shape)
    
    return (x_train, y_train), (x_test, y_test),(x_val,y_val)

### Main

In [16]:
"""
text classification using
keras-fasttext 
"""
from __future__ import print_function
import numpy as np

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import CSVLogger

# import data_reader_yahoo as data_reader_yahoo
import os
import math


###############################################
# user params
###############################################

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-s", "--seed", help="random seed")
parser.add_argument("-sd", "--savedir", help="saving directory")
parser.add_argument("-f", "--datafile", help="data file", required=True)
parser.add_argument("-k", "--topk", help="top k models to save", default=10)
parser.add_argument("-g","--gpu",default=0)

class Bunch:
  def __init__(self, **entries):
    self.__dict__.update(entries)

# args = parser.parse_args()
args = Bunch(
    seed=10,
    savedir='./output/yahoo_answers_csv_imbalance3/DS1/baseline/run_1',
    datafile='./data/yahoo_answers_csv_imbalance3/DS1',
    topK=10,
    gpu=0
)
save_dir = args.savedir
datafile = args.datafile
seed     = int(args.seed)
gpuId    = args.gpu
top_k    = 1

#add gpu target
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# tf.config.set_visible_devices([], 'GPU')

In [17]:
# del os.environ["CUDA_VISIBLE_DEVICES"]

# add destination dir:
print ("output dir",save_dir)
if not os.path.exists(save_dir):
    print("adding saving directory")
    os.makedirs(save_dir)
# Set random seed
if seed is not None:
    print('Setting seed.')
    import tensorflow as tf
    tf.random.set_seed(seed)
    np.random.seed(seed)

###############################################
# MODEL params
###############################################
# Set parameters:
# ngram_range = 2 will add bi-grams features - we don't use this feature
ngram_range = 1
max_features = 10000
maxlen = 1014
batch_size = 16
embedding_dims = 50
epochs = 1
#params
num_classes =10

output dir ./output/yahoo_answers_csv_imbalance3/DS1/baseline/run_1
Setting seed.


In [18]:

snapshot_window_size = int(math.ceil(epochs/top_k))
print('Loading data...')
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
(x_train, y_train), (x_test, y_test) ,(x_val,y_val)= load_data_yahoo_ans(datafile,
                                                                             max_words=max_features,
                                                                             max_len=maxlen)

print (type(x_train))
print (type(y_train))
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(
    np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(
    np.mean(list(map(len, x_test)), dtype=int)))

###############################################
# MODEL definitions
###############################################
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.

    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]

    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences


if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(
        np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(
        np.mean(list(map(len, x_test)), dtype=int)))

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
to read
train len = 683200
test len  = 2440
val len   = 26840
to process
to tokenize
unique tokens: 668410
to split
New train SHAPE
(683200, 1014)
(683200, 9)
New test SHAPE
(2440, 1014)
(2440, 9)
New train SHAPE
(2440, 1014)
(2440, 9)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
683200 train sequences
2440 test sequences
Average train sequence length: 1014
Average test sequence length: 1014
Pad sequences (samples x time)
x_train shape: (683200, 1014)
x_test shape: (2440, 1014)


In [19]:
devices = tf.config.list_physical_devices('GPU')
print(devices)
tf.config.set_visible_devices(devices[0], 'GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-07-01 16:25:06.321796: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:1e:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-07-01 16:25:06.325249: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0


In [20]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [21]:
# from tensorflow.python.keras import backend as K
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto())
# K.set_session(sess)
# os.environ.get("CUDA_VISIBLE_DEVICES", None)

In [26]:
with tf.device('/device:GPU:0'), tf.compat.v1.Session(config=tf.compat.v1.ConfigProto()):
    print('Build model...')
    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen))

    # we add a GlobalAveragePooling1D, which will average the embeddings
    # of all words in the document
    model.add(GlobalAveragePooling1D())

    # We project onto a single unit output layer, and squash it with a sigmoid:
    #model.add(Dense(2, activation='sigmoid'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    #model.compile(loss='binary_crossentropy',
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    cp = IcpClassifier(model)


    ###############################################
    # PREDICTION AND OUPUT
    ###############################################
    ## add logs
    # Training log writer
    logfile   = '{}/callback_training_log.csv'.format(save_dir)
    csvlog    =  CSVLogger(logfile, separator=',', append=False)
    callbacks = [csvlog]

    history = cp.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_val, y_val),
            callbacks=callbacks
          )

    # Save the weights
    model.save_weights(save_dir+'model_weights.h5')

    # Save the model architecture
    with open(save_dir+'model_architecture.json', 'w') as f:
        f.write(model.to_json())

2021-07-01 16:28:44.663385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:1e:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-07-01 16:28:44.664682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2021-07-01 16:28:44.664735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-07-01 16:28:44.664747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2021-07-01 16:28:44.664757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2021-07-01 16:28:44.666044: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1418] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 9659 MB memory) -> physical GPU (device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:1e:00.0, compu

Build model...
Train on 683200 samples, validate on 26840 samples
682944/683200 [============================>.] - ETA: 0s - loss: 1.2852 - accuracy: 0.6015

/home/michael.me/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


683200/683200 [==============================] - 122s 179us/sample - loss: 1.2852 - accuracy: 0.6016 - val_loss: 1.0451 - val_accuracy: 0.6891


In [27]:
history.history

{'loss': [1.2851576614121643],
 'accuracy': [0.6015808],
 'val_loss': [1.045074704062921],
 'val_accuracy': [0.68908346]}

In [28]:
# Save training log
print('Saving training log...')
train_error    = history.history['loss']
valid_accuracy = history.history['val_accuracy']

# Save index for combination
c = [str(i) for i in range(num_classes)]
header = ','.join(c) + '\n'
print('Writing index file and predict files...')
indexfile = '{}/index.csv'.format(save_dir)
f = open(indexfile, 'w')
top_x = []


#we have a single file for prediciton
# to check later.
x=0
name = 'prediction_{:04d}.csv'.format(x+1)
#use the last model to predict
weight = valid_accuracy[epochs-1]
f.write('{},{}\n'.format(name, weight))
predicts = model.predict(x_test)
# Save predicts
predictfile = '{}/prediction_{:04d}.csv'.format(save_dir, x+1)
f1 = open(predictfile,'w')
f1.write(header)
np.savetxt(f1, predicts, delimiter=",")
f1.close()

    
# Save targets
print('Saving target file...')
targetfile = '{}/target.csv'.format(save_dir)
f2 = open(targetfile,'w')
f2.write(header)
np.savetxt(f2, y_test, delimiter=",")
f2.close()

Saving training log...
Writing index file and predict files...


TypeError: _maybe_load_initial_epoch_from_ckpt() takes 2 positional arguments but 3 were given

## Conformal Prediction